#upload kaggle json file





In [1]:
!pip install kaggle
from google.colab import files 
files.upload() # upload json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d gakowsher/banglamct7-bangla-multiclass-text-dataset-7-tags

 99% 331M/335M [00:08<00:00, 64.0MB/s]
100% 335M/335M [00:08<00:00, 39.0MB/s]


In [3]:
from zipfile import ZipFile
file_name= "banglamct7-bangla-multiclass-text-dataset-7-tags.zip"

with ZipFile(file_name ,'r') as zip:
  zip.extractall()
  print('Done')

Done


#Import Libraries

In [4]:
import pandas as pd
import numpy as np
import os
from google.colab import output


In [5]:
import re
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Activation,Embedding,Flatten,Bidirectional,MaxPooling2D, Conv1D, MaxPooling1D

from keras import regularizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical


from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

  Created wheel for py-bangla-stemmer: filename=py_bangla_stemmer-0.5.1-py3-none-any.whl size=17351 sha256=752a4af1cb70603688dbb7429bb0190d2bda156e9b702593a836e82fb65d4fb6
  Stored in directory: /root/.cache/pip/wheels/30/bc/a9/37aed1bed9523e97e10d89ff21a92d82026b88974be8931336
Successfully built py-bangla-stemmer


#Read dataset

In [6]:
train_data=pd.read_csv('/content/BanglaMCT7/train.csv')
test_data = pd.read_csv('/content/BanglaMCT7/test.csv')

In [7]:
train_data.head()

,category,text,cleanText
0,economy,বাংলাদেশে সফররত বিশ্ব বাণিজ্য সংস্থা (ডব্লিউট...,বাংলাদেশে সফররত বিশ্ব বাণিজ্য সং...
1,sports,ইউরোপিয়ান ফুটবলে আজ ঝড় বয়ে যাচ্ছে। একদিকে সংবা...,ইউরোপিয়ান ফুটবলে বয়ে একদিকে সংবা...
2,international,বিজেপির নেতৃত্বে ভারতের উত্তর প্রদেশ থেকে শুরু...,বিজেপির নেতৃত্বে ভারতের উত্তর প্...
3,state,নারায়ণগঞ্জের সোনারগাঁ উপজেলার একটি সরকারি প্র...,নারায়ণগঞ্জের সোনারগাঁ উপজেলার সরকা...
4,sports,"‘ছাই থেকে ছাই, ধুলো থেকে ধুলো...’ আধ্যাত্মিক ক...",ছাই ছাই ধুলো ধুলো আধ্যাত্মিক ...


In [8]:
train_data.text.iloc[5]

'খুদে ব্লগ লেখার জনপ্রিয় সাইট টুইটার এখন বাংলা ভাষায় দেখা যাবে। টুইটারের মূল ওয়েবসাইটের পাশাপাশি মোবাইল ওয়েবসাইট(mobile.twitter.com), অ্যান্ড্রয়েড ও আইওএস অ্যাপ্লিকেশনে (অ্যাপ) বাংলা সমর্থনের সুবিধা চালু করেছে। এর ফলে সম্পূর্ণ বাংলায় বার্তা (টুইট) লেখা ও পড়া যাবে। বাংলা ছাড়াও সম্প্রতি ভিয়েতনামিস, চেক, রোমানিয়ান ও ইউক্রেনিয়ান ভাষা যুক্ত হয়েছে টুইটারে। বিভিন্ন ভাষাভাষীর স্বেচ্ছাসেবকদের অনুবাদ করা এসব ভাষা এর আগে শুধু অনুবাদকাজের সঙ্গে যুক্ত স্বেচ্ছাসেবকদের জন্য উন্মুক্ত ছিল।গতকাল মঙ্গলবার থেকে এসব ভাষার সুবিধা সাধারণ ব্যবহারকারীদের জন্য উন্মুক্ত করা হয়েছে। বাংলা কিংবা অন্যান্য ভাষা নির্বাচন করতে টুইটারের সেটিংস থেকে ভাষা অপশনে গিয়ে পছন্দের ভাষা বেছে নিতে হবে।চাইলে যে কেউ টুইটারের এ ভাষা অনুবাদেও যুক্ত হতে পারেন। এতে নির্দিষ্ট শব্দ কিংবা লাইনের সঠিক অর্থ লিখে জমা দেওয়া যাবে। সেখান থেকে নির্বাচিত এবং সঞ্চালক কর্তৃক পরীক্ষিত হওয়ার পর সেটি মূল টুইটারে বাংলা ভাষায় যুক্ত হবে। আগ্রহীরা (translate.twitter.com) ঠিকানায় গিয়ে বাংলা অনুবাদে যুক্ত হতে পারেন।উল্লেখ্য, ২০০৬ সালে মাত্র ১৪০ শব্দের মধ্

In [9]:
train_data.cleanText.iloc[5]

'  খুদে    ব্লগ    লেখার    জনপ্রিয়    সাইট    টুইটার    বাংলা    ভাষায়    টুইটারের    মূল    ওয়েবসাইটের    পাশাপাশি    মোবাইল    ওয়েবসাইট    অ্যান্ড্রয়েড    আইওএস    অ্যাপ্লিকেশনে    অ্যাপ    বাংলা    সমর্থনের    সুবিধা    চালু    সম্পূর্ণ    বাংলায়    বার্তা    টুইট    লেখা    পড়া    বাংলা    ছাড়াও    ভিয়েতনামিস    চেক    রোমানিয়ান    ইউক্রেনিয়ান    ভাষা    যুক্ত    টুইটারে    বিভিন্ন    ভাষাভাষীর    স্বেচ্ছাসেবকদের    অনুবাদ    এসব    ভাষা    অনুবাদকাজের    যুক্ত    স্বেচ্ছাসেবকদের    উন্মুক্ত    গতকাল    মঙ্গলবার    এসব    ভাষার    সুবিধা    সাধারণ    ব্যবহারকারীদের    উন্মুক্ত    বাংলা    অন্যান্য    ভাষা    নির্বাচন    টুইটারের    সেটিংস    ভাষা    অপশনে    পছন্দের    ভাষা    বেছে    চাইলে    টুইটারের    ভাষা    অনুবাদেও    যুক্ত    নির্দিষ্ট    শব্দ    লাইনের    সঠিক    অর্থ    লিখে    জমা    নির্বাচিত    সঞ্চালক    কর্তৃক    পরীক্ষিত    মূল    টুইটারে    বাংলা    ভাষায়    যুক্ত    আগ্রহীরা    ঠিকানায়    বাংলা    অনুবাদে    যুক্ত    উল্লেখ্য    সালে    শব্দের    খুদে    ব্লগ 

In [10]:
test_data.head()

,category,text,cleanText
0,education,"অধ্যায়-২প্রিয় পরীক্ষার্থী, আজ তথ্য ও যোগাযোগ প...",অধ্যায় প্রিয় পরীক্ষার্থী তথ্য যো...
1,sports,"স্পিন, স্পিন, স্পিন। লর্ডসে ইয়াসির শাহ্‌র লেগ ...",স্পিন স্পিন স্পিন লর্ডসে ইয়াসির ...
2,international,সিরিয়ার উত্তর-পশ্চিমাঞ্চলে গতকাল রোববার বিমান...,সিরিয়ার উত্তর পশ্চিমাঞ্চলে গতকাল ...
3,international,দক্ষিণ আফ্রিকার কেপটাউন বিশ্ববিদ্যালয়ের (ইউসিট...,দক্ষিণ আফ্রিকার কেপটাউন বিশ্ববিদ্যা...
4,technology,সহজে ডিজিটাল আর্থিক সেবা দিতে অ্যাপ ও ডিজিটাল ...,সহজে ডিজিটাল আর্থিক সেবা অ্যাপ ...


In [11]:
train_data

,category,text,cleanText
0,economy,বাংলাদেশে সফররত বিশ্ব বাণিজ্য সংস্থা (ডব্লিউট...,বাংলাদেশে সফররত বিশ্ব বাণিজ্য সং...
1,sports,ইউরোপিয়ান ফুটবলে আজ ঝড় বয়ে যাচ্ছে। একদিকে সংবা...,ইউরোপিয়ান ফুটবলে বয়ে একদিকে সংবা...
2,international,বিজেপির নেতৃত্বে ভারতের উত্তর প্রদেশ থেকে শুরু...,বিজেপির নেতৃত্বে ভারতের উত্তর প্...
3,state,নারায়ণগঞ্জের সোনারগাঁ উপজেলার একটি সরকারি প্র...,নারায়ণগঞ্জের সোনারগাঁ উপজেলার সরকা...
4,sports,"‘ছাই থেকে ছাই, ধুলো থেকে ধুলো...’ আধ্যাত্মিক ক...",ছাই ছাই ধুলো ধুলো আধ্যাত্মিক ...
...,...,...,...
98878,education,সহকারী অধ্যাপক টিচার্স ট্রেনিং কলেজ (মহিলা)...,সহকারী অধ্যাপক টিচার্স ট্রেনিং ক...
98879,entertainment,অক্টোবরের ১১ তারিখে ৭৫ বছর পূর্ণ করবেন অমিতাভ ...,অক্টোবরের তারিখে বছর পূর্ণ অমিতা...
98880,technology,কাজী নাঈম আল রশিদ। সফটওয়্যার প্রকৌশলী। কাজ করে...,কাজী নাঈম রশিদ সফটওয়্যার প্রকৌশল...
98881,technology,ইয়াহুকে মার্কিন টেলিকম প্রতিষ্ঠান ভেরাইজন কিনে...,ইয়াহুকে মার্কিন টেলিকম প্রতিষ্ঠান ...


In [12]:
test_data

,category,text,cleanText
0,education,"অধ্যায়-২প্রিয় পরীক্ষার্থী, আজ তথ্য ও যোগাযোগ প...",অধ্যায় প্রিয় পরীক্ষার্থী তথ্য যো...
1,sports,"স্পিন, স্পিন, স্পিন। লর্ডসে ইয়াসির শাহ্‌র লেগ ...",স্পিন স্পিন স্পিন লর্ডসে ইয়াসির ...
2,international,সিরিয়ার উত্তর-পশ্চিমাঞ্চলে গতকাল রোববার বিমান...,সিরিয়ার উত্তর পশ্চিমাঞ্চলে গতকাল ...
3,international,দক্ষিণ আফ্রিকার কেপটাউন বিশ্ববিদ্যালয়ের (ইউসিট...,দক্ষিণ আফ্রিকার কেপটাউন বিশ্ববিদ্যা...
4,technology,সহজে ডিজিটাল আর্থিক সেবা দিতে অ্যাপ ও ডিজিটাল ...,সহজে ডিজিটাল আর্থিক সেবা অ্যাপ ...
...,...,...,...
98879,sports,১৯৫৯ সালে ইউরোপিয়ান ক্লাব কাপে জাতীয় দল পাঠান...,সালে ইউরোপিয়ান ক্লাব কাপে জাতীয়...
98880,entertainment,গানের দল সহজিয়ার প্রথম অ্যালবাম ‘রঙমিস্ত্রী’র ...,গানের সহজিয়ার প্রথম অ্যালবাম রঙম...
98881,international,ফেসবুকের ৮১ হাজার ব্যবহারকারীর তথ্য চুরি করে স...,ফেসবুকের হাজার ব্যবহারকারীর তথ্য ...
98882,economy,পেট্রোল পাম্পে তেল বা সিএনজির বিল বিকাশের মাধ...,পেট্রোল পাম্পে তেল সিএনজির বিল ...


In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98883 entries, 0 to 98882
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   category   98883 non-null  object
 1   text       98883 non-null  object
 2   cleanText  98883 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB


In [14]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98884 entries, 0 to 98883
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   category   98884 non-null  object
 1   text       98884 non-null  object
 2   cleanText  98884 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB


In [15]:
train_data.category.value_counts()

sports           15672
international    15540
state            15362
entertainment    15198
economy          14430
education        12674
technology       10007
Name: category, dtype: int64

In [16]:
test_data.category.value_counts()

sports           15883
international    15314
entertainment    15261
state            15183
economy          14488
education        12818
technology        9937
Name: category, dtype: int64

In [17]:
print(train_data.isna().sum())

category     0
text         0
cleanText    0
dtype: int64


In [18]:
print(test_data.isna().sum())

category     0
text         0
cleanText    0
dtype: int64


In [19]:
train_data['category'].unique()

array(['economy', 'sports', 'international', 'state', 'technology',
       'entertainment', 'education'], dtype=object)

In [20]:
train_data.head()

,category,text,cleanText
0,economy,বাংলাদেশে সফররত বিশ্ব বাণিজ্য সংস্থা (ডব্লিউট...,বাংলাদেশে সফররত বিশ্ব বাণিজ্য সং...
1,sports,ইউরোপিয়ান ফুটবলে আজ ঝড় বয়ে যাচ্ছে। একদিকে সংবা...,ইউরোপিয়ান ফুটবলে বয়ে একদিকে সংবা...
2,international,বিজেপির নেতৃত্বে ভারতের উত্তর প্রদেশ থেকে শুরু...,বিজেপির নেতৃত্বে ভারতের উত্তর প্...
3,state,নারায়ণগঞ্জের সোনারগাঁ উপজেলার একটি সরকারি প্র...,নারায়ণগঞ্জের সোনারগাঁ উপজেলার সরকা...
4,sports,"‘ছাই থেকে ছাই, ধুলো থেকে ধুলো...’ আধ্যাত্মিক ক...",ছাই ছাই ধুলো ধুলো আধ্যাত্মিক ...


#Data Processing:

In [22]:
label_dict

{'economy': 0,
 'education': 6,
 'entertainment': 5,
 'international': 2,
 'sports': 1,
 'state': 3,
 'technology': 4}

In [21]:
labels= train_data.category.unique()
label_dict={}

for index,possible_label in enumerate(labels):
  label_dict[possible_label]=index

train_data['label']=train_data.category.replace(label_dict)
test_data['label']=test_data.category.replace(label_dict)

train_data.head()

,category,text,cleanText,label
0,economy,বাংলাদেশে সফররত বিশ্ব বাণিজ্য সংস্থা (ডব্লিউট...,বাংলাদেশে সফররত বিশ্ব বাণিজ্য সং...,0
1,sports,ইউরোপিয়ান ফুটবলে আজ ঝড় বয়ে যাচ্ছে। একদিকে সংবা...,ইউরোপিয়ান ফুটবলে বয়ে একদিকে সংবা...,1
2,international,বিজেপির নেতৃত্বে ভারতের উত্তর প্রদেশ থেকে শুরু...,বিজেপির নেতৃত্বে ভারতের উত্তর প্...,2
3,state,নারায়ণগঞ্জের সোনারগাঁ উপজেলার একটি সরকারি প্র...,নারায়ণগঞ্জের সোনারগাঁ উপজেলার সরকা...,3
4,sports,"‘ছাই থেকে ছাই, ধুলো থেকে ধুলো...’ আধ্যাত্মিক ক...",ছাই ছাই ধুলো ধুলো আধ্যাত্মিক ...,1


In [23]:
def remove_space(x):
  x= re.sub('\s+',' ',x)
  return x

train_data['cleanText']= train_data['cleanText'].apply(remove_space)

In [24]:
from collections import Counter
result = len(set(Counter(" ".join(train_data['cleanText'].values.tolist()).split(" ")).items()))
result

464249

In [25]:
# from bangla_stemmer.stemmer import stemmer
# def stem_text (x):
#   stmr = stemmer.BanglaStemmer()
#   words=x.split(' ')
#   stm = stmr.stem(words)
#   words=(' ').join(stm)
#   output.clear()
#   return words

# train_data['cleanText']=train_data['cleanText'].apply(stem_text)




In [26]:
# train_data.to_csv('train_data_stemmer.csv')

#Naive Bayes

In [28]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer()
tf_train=vect.fit_transform(train_data['cleanText'])  #train the vectorizer, build the vocablury
tf_test=vect.transform(test_data['cleanText'])  #get same encodings on test data as of vocabulary built

model.fit(X=tf_train,y=train_data['category'])

expected = test_data['category']
predicted=model.predict(tf_test)

#Classification Scores and Confusion Matrix

In [29]:
from sklearn import metrics

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

               precision    recall  f1-score   support

      economy       0.86      0.82      0.84     14488
    education       0.86      0.83      0.84     12818
entertainment       0.89      0.92      0.90     15261
international       0.88      0.88      0.88     15314
       sports       0.98      0.92      0.95     15883
        state       0.80      0.84      0.82     15183
   technology       0.79      0.86      0.82      9937

     accuracy                           0.87     98884
    macro avg       0.87      0.87      0.87     98884
 weighted avg       0.87      0.87      0.87     98884

[[11902   286   105   253    22   872  1048]
 [  309 10604   318    76    50  1065   396]
 [   40   180 13994   373   134   387   153]
 [  274   160   506 13509    87   346   432]
 [   95    53   292   420 14643   276   104]
 [  750   730   296   498    27 12749   133]
 [  466   330   165   278    35   146  8517]]


#Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(n_jobs=1, C=1e5 , multi_class='multinomial')
logreg.fit(X=tf_train,y=train_data['category'])

expected = test_data['category']
predicted=logreg.predict(tf_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#Classification Scores and Confusion Matrix

In [31]:
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

               precision    recall  f1-score   support

      economy       0.87      0.86      0.86     14488
    education       0.88      0.89      0.89     12818
entertainment       0.91      0.92      0.92     15261
international       0.90      0.89      0.89     15314
       sports       0.95      0.96      0.96     15883
        state       0.86      0.85      0.85     15183
   technology       0.84      0.84      0.84      9937

     accuracy                           0.89     98884
    macro avg       0.89      0.89      0.89     98884
 weighted avg       0.89      0.89      0.89     98884

[[12407   298    88   251   110   714   620]
 [  224 11414   185    70   154   510   261]
 [   71   140 14031   325   214   357   123]
 [  241   134   378 13645   160   386   370]
 [   33    65   218   165 15258    82    62]
 [  735   581   304   414    82 12912   155]
 [  617   307   149   358    66   132  8308]]
